In [1]:
# use 2/3 of HC as training set and remaining HC, and disorders as test set.
# 5/29/2021.
# split training, and testing for males and female separately.
# 5/29/2021

import os
from os import path
import pandas as pd

# data_dir="/home/xin/Dropbox/BrainImaging_ML/data"
data_dir="C:/Users/Xin/OneDrive - Drexel University/BrainImaging_ML/data"
# data_dir="C:/Users/niuxi/Dropbox/BrainImaging_ML/data"

input_dir1 = "out01_robustreg_behav_hc2_stdz_age_reverse_fa"
input_dir2 = "out02_robustreg_behav_hc2_stdz_age_reverse_fa_notract_scale_thresh.05"
output_dir = "out03_age_prediction_hc2_stdz_age_reverse_notract_fa_scale_thresh.05"

hc_label = "mental_hc2"

# cluster_col_index = 'cluster_km4'
cluster_col_index = 'cluster_gmmEEE4'

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

cluster_data = pd.read_csv(input_dir2+'/out02_coefs_cluster_R_withintercept.csv', index_col = 0)
# cluster_data = pd.read_csv(input_dir2+'/out02_coefs_cluster_sklearn.csv', index_col = 0)

behav_feature = pd.read_csv(input_dir1+'/out01_behav_feature_all_raw.csv', index_col = 0)
behav_feature['Sex'] = behav_feature['Sex'].apply({'F':0, 'M':1}.get)
subject_id = behav_feature['SUBJID'].to_frame()

##  extract train index in HCs:
## 5/29/2021
behav_feature_hc = pd.read_csv(input_dir1+'/out01_behav_feature_hc.csv', index_col = 0)
hc_subject_train = behav_feature_hc.loc[behav_feature_hc['Train_index']==True, 'SUBJID']

behav_feature.insert(loc = 1, column = 'Train_index', value = False)
behav_feature.loc[behav_feature['SUBJID'].isin(hc_subject_train), 'Train_index'] = True

# remove some tract FA features (index 1:18) that are highly correlated to label FA features.
start_ind = 4
drop_labels = behav_feature.iloc[:,(start_ind+116+50):(start_ind+116+70)].columns
behav_feature.drop(columns = drop_labels, inplace = True)

cluster_data.columns.values[0] = "feature"
cluster_data['feature'] = behav_feature.iloc[:,(start_ind+1):].columns
cluster_data.columns.values[[3,5,6]] = [ 'age$^2$', 'age*gender', 'age$^2$*gender']

# newly merged file (SUBJID and bblid) with pandas:
pnc_behav_mental = pd.read_csv(data_dir+"/pyresult01_pnc_behav_mental4.csv", delimiter = ",")
pnc_behav_mental = pnc_behav_mental.astype({'SUBJID': 'int64'})


print(pnc_behav_mental.shape)

pnc_behav_mental = subject_id.merge(pnc_behav_mental, how = "left", left_on = "SUBJID", right_on = "SUBJID")
print(pnc_behav_mental.shape)

pnc_behav_mental['Sex'] = pnc_behav_mental['Sex'].apply({'F':0, 'M':1}.get)

# subject_info_merge.to_csv("data/script01_disorders_subject_info_merge.csv", sep = ",")
      
print(pnc_behav_mental.head())


(8519, 102)
(610, 102)
         SUBJID     bblid  Sex  age_at_cnb Med_Rating  pcet_acc2  pcpt_t_tp  \
0  600009963128  117129.0    0         9.0          1     2.7143       47.0   
1  600018902293  127386.0    0        15.0          1     2.1538       59.0   
2  600038720566   94599.0    0        11.0          1     0.4583       56.0   
3  600039015048  101501.0    1        11.0          1     3.0400       54.0   
4  600039665619  101368.0    1         9.0          1     0.3958       28.0   

   lnb_tp2  pwmt_kiwrd_tot  pfmt_ifac_tot  ...  smry_hal_vh  smry_hal_oh  \
0      6.0            39.0           34.0  ...            0            0   
1      9.0            40.0           35.0  ...            0            0   
2      8.0            39.0           35.0  ...            0            0   
3      8.0            40.0           30.0  ...            0            0   
4      9.0            30.0           24.0  ...            0            0   

   smry_hal_th  smry_beh_cat  smry_add  smry_

In [2]:
behav_feature

,SUBJID,Train_index,Sex,age_at_cnb,names,Left Precentral,Right Precentral,Left Superior Frontal,Right Superior Frontal,Left Superior Frontal Orbital,...,Superior.longitudinal.fasciculus.R(label),Superior.longitudinal.fasciculus.L(label),Superior.fronto-occipital.fasciculus.(could.be.a.part.of.anterior.internal.capsule).R(label),Superior.fronto-occipital.fasciculus.(could.be.a.part.of.anterior.internal.capsule).L(label),Inferior.fronto-occipital.fasciculus.R(label),Inferior.fronto-occipital.fasciculus.L(label),Uncinate.fasciculus.R(label),Uncinate.fasciculus.L(label),Tapetum.R(label),Superior.longitudinal.fasciculus.(temporal.part).R(tract)
0,600009963128,True,0,9.0,600009963128,0.496769,0.492516,0.469744,0.472897,0.555330,...,-0.449311,-0.438606,-0.498259,-0.495606,-0.434996,-0.428980,-0.429856,-0.441601,-0.400445,-0.596944
1,600018902293,False,0,15.0,600018902293,0.449852,0.416304,0.409503,0.405678,0.480874,...,-0.437091,-0.439121,-0.424826,-0.396323,-0.447932,-0.440448,-0.444648,-0.433922,-0.417638,-0.530693
2,600038720566,False,0,11.0,600038720566,0.537039,0.500568,0.393680,0.396776,0.462140,...,-0.405389,-0.395929,-0.429126,-0.427535,-0.415323,-0.383320,-0.397618,-0.381825,-0.427577,-0.506131
3,600039015048,True,1,11.0,600039015048,0.492985,0.456270,0.445612,0.430054,0.530365,...,-0.403509,-0.385902,-0.413289,-0.459313,-0.399568,-0.387467,-0.368935,-0.375250,-0.398979,-0.477383
4,600039665619,False,1,9.0,600039665619,0.518909,0.489561,0.511222,0.472811,0.614461,...,-0.402340,-0.396500,-0.465295,-0.453903,-0.393954,-0.384343,-0.420502,-0.406804,-0.410951,-0.489960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,609903870257,False,1,14.0,609903870257,0.516216,0.501418,0.458626,0.472434,0.479979,...,-0.435713,-0.430831,-0.419857,-0.452930,-0.418786,-0.430110,-0.436948,-0.430470,-0.467175,-0.572417
606,609910470154,False,0,19.0,609910470154,0.470289,0.460937,0.413662,0.417567,0.478178,...,-0.403931,-0.405431,-0.416981,-0.408175,-0.386828,-0.403334,-0.374796,-0.380701,-0.407016,-0.501234
607,609919910881,False,1,15.0,609919910881,0.483536,0.431535,0.396026,0.376200,0.464298,...,-0.440654,-0.447909,-0.475961,-0.461891,-0.443016,-0.430285,-0.455808,-0.427278,-0.430173,-0.541160
608,609987705265,False,1,15.0,609987705265,0.546239,0.524109,0.491601,0.512870,0.509529,...,-0.449078,-0.443418,-0.436006,-0.443410,-0.483405,-0.469088,-0.432302,-0.407773,-0.396675,-0.539863


In [3]:
pnc_behav_mental

,SUBJID,bblid,Sex,age_at_cnb,Med_Rating,pcet_acc2,pcpt_t_tp,lnb_tp2,pwmt_kiwrd_tot,pfmt_ifac_tot,...,smry_hal_vh,smry_hal_oh,smry_hal_th,smry_beh_cat,smry_add,smry_odd,smry_con,mental_hc1,mental_hc2,mental_hc3
0,600009963128,117129.0,0,9.0,1,2.7143,47.0,6.0,39.0,34.0,...,0,0,0,0,0,0,0,False,True,True
1,600018902293,127386.0,0,15.0,1,2.1538,59.0,9.0,40.0,35.0,...,0,0,0,2,2,1,0,False,False,False
2,600038720566,94599.0,0,11.0,1,0.4583,56.0,8.0,39.0,35.0,...,0,0,2,4,4,4,2,False,False,False
3,600039015048,101501.0,1,11.0,1,3.0400,54.0,8.0,40.0,30.0,...,0,0,0,0,0,0,0,True,True,True
4,600039665619,101368.0,1,9.0,1,0.3958,28.0,9.0,30.0,24.0,...,0,0,0,3,3,0,0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,609903870257,104511.0,1,14.0,0,2.0000,54.0,10.0,40.0,23.0,...,0,0,0,4,1,4,1,False,False,False
606,609910470154,91489.0,0,19.0,0,2.5946,50.0,9.0,37.0,35.0,...,0,0,0,4,4,0,0,False,False,False
607,609919910881,112249.0,1,15.0,1,2.3529,57.0,8.0,38.0,31.0,...,0,0,0,4,1,4,1,False,False,False
608,609987705265,126389.0,1,15.0,1,2.4516,50.0,10.0,37.0,30.0,...,0,0,0,1,0,1,0,True,True,True


In [4]:
cluster_data.columns.values[0] = "feature"
# cluster_data

In [5]:

# age prediction for each gmm cluster:
cluster_data_gmm = cluster_data.loc[cluster_data[cluster_col_index]<10,:]

cluster_data_gmm[cluster_col_index].unique()

array([1., 2., 3., 4.])

In [6]:
disorder_count = pnc_behav_mental[['smry_mood_cat', 'smry_man', 'smry_dep', 'smry_eat_cat', 
                                     'smry_bul', 'smry_ano', 'smry_anx_cat', 'smry_gad', 'smry_sep', 
                                     'smry_phb', 'smry_soc', 'smry_pan', 'smry_agr',
                                     'smry_ocd', 'smry_ptd', 'smry_psy_cat', 'smry_del', 'smry_hal',
                                     'smry_hal_av', 'smry_hal_as', 'smry_hal_vh', 'smry_hal_oh',
                                     'smry_hal_th', 'smry_beh_cat', 'smry_add', 'smry_odd', 'smry_con']]==4

disorder_count.sum(axis = 0)

smry_mood_cat     74
smry_man           7
smry_dep          70
smry_eat_cat       9
smry_bul           3
smry_ano           6
smry_anx_cat     303
smry_gad           8
smry_sep          21
smry_phb         185
smry_soc         142
smry_pan           6
smry_agr          30
smry_ocd          19
smry_ptd          71
smry_psy_cat      36
smry_del           0
smry_hal           0
smry_hal_av        0
smry_hal_as        0
smry_hal_vh        0
smry_hal_oh        0
smry_hal_th        0
smry_beh_cat     252
smry_add         106
smry_odd         206
smry_con          52
dtype: int64

In [7]:
# subjects without comorbidity:
disorder_count = pnc_behav_mental[['smry_mood_cat', 'smry_man', 'smry_dep', 'smry_eat_cat', 
                                     'smry_bul', 'smry_ano', 'smry_anx_cat', 'smry_gad', 'smry_sep', 
                                     'smry_phb', 'smry_soc', 'smry_pan', 'smry_agr',
                                     'smry_ocd', 'smry_ptd', 'smry_psy_cat', 'smry_del', 'smry_hal',
                                     'smry_hal_av', 'smry_hal_as', 'smry_hal_vh', 'smry_hal_oh',
                                     'smry_hal_th','smry_beh_cat', 'smry_add', 'smry_odd', 'smry_con']]==4

(disorder_count['smry_con']&~disorder_count['smry_odd']).sum()

9

In [8]:
(disorder_count['smry_sep']&~disorder_count['smry_ptd']).sum()

16

In [9]:
disorders = ['smry_ptd', 'smry_dep', 'smry_phb', 'smry_soc', 'smry_add', 'smry_odd']
# subject_info_columns = ['SUBJID', 'Sex', 'age_at_cnb']
subject_info_columns = ['SUBJID']

subject_info_hc = pnc_behav_mental.loc[pnc_behav_mental[hc_label]==True, subject_info_columns]
subject_info_hc['group'] = 'HC'
print(subject_info_hc.shape)

subject_info_group = [subject_info_hc]
for dis in disorders:
    subject_info_dis = pnc_behav_mental.loc[pnc_behav_mental[dis]==4, subject_info_columns]
    subject_info_dis['group'] = dis
    print(dis)
    print(subject_info_dis.shape)
    subject_info_group.append(subject_info_dis)
    
subject_info_group = pd.concat(subject_info_group, axis = 0)
subject_info_group   

(166, 2)
smry_ptd
(71, 2)
smry_dep
(70, 2)
smry_phb
(185, 2)
smry_soc
(142, 2)
smry_add
(106, 2)
smry_odd
(206, 2)


,SUBJID,group
0,600009963128,HC
3,600039015048,HC
14,600110937794,HC
17,600173623767,HC
24,600294591042,HC
...,...,...
602,609812189682,smry_odd
603,609849166759,smry_odd
605,609903870257,smry_odd
607,609919910881,smry_odd


In [10]:
behav_feature_group = behav_feature.merge(subject_info_group, on = 'SUBJID', how = 'inner')
behav_feature_group['Sex'] = (behav_feature_group['Sex']+1)/2
behav_feature_group['Sex'] = behav_feature_group['Sex'].astype('int32')

# move group to the front of features:
group = behav_feature_group.pop('group')
behav_feature_group.insert(loc = 1, column = 'group', value = group)
behav_feature_group.loc[behav_feature_group['Train_index']==True, 'group']='HC_train'
behav_feature_group.pop('Train_index')
behav_feature_group

,SUBJID,group,Sex,age_at_cnb,names,Left Precentral,Right Precentral,Left Superior Frontal,Right Superior Frontal,Left Superior Frontal Orbital,...,Superior.longitudinal.fasciculus.R(label),Superior.longitudinal.fasciculus.L(label),Superior.fronto-occipital.fasciculus.(could.be.a.part.of.anterior.internal.capsule).R(label),Superior.fronto-occipital.fasciculus.(could.be.a.part.of.anterior.internal.capsule).L(label),Inferior.fronto-occipital.fasciculus.R(label),Inferior.fronto-occipital.fasciculus.L(label),Uncinate.fasciculus.R(label),Uncinate.fasciculus.L(label),Tapetum.R(label),Superior.longitudinal.fasciculus.(temporal.part).R(tract)
0,600009963128,HC_train,0,9.0,600009963128,0.496769,0.492516,0.469744,0.472897,0.555330,...,-0.449311,-0.438606,-0.498259,-0.495606,-0.434996,-0.428980,-0.429856,-0.441601,-0.400445,-0.596944
1,600018902293,smry_soc,0,15.0,600018902293,0.449852,0.416304,0.409503,0.405678,0.480874,...,-0.437091,-0.439121,-0.424826,-0.396323,-0.447932,-0.440448,-0.444648,-0.433922,-0.417638,-0.530693
2,600038720566,smry_add,0,11.0,600038720566,0.537039,0.500568,0.393680,0.396776,0.462140,...,-0.405389,-0.395929,-0.429126,-0.427535,-0.415323,-0.383320,-0.397618,-0.381825,-0.427577,-0.506131
3,600038720566,smry_odd,0,11.0,600038720566,0.537039,0.500568,0.393680,0.396776,0.462140,...,-0.405389,-0.395929,-0.429126,-0.427535,-0.415323,-0.383320,-0.397618,-0.381825,-0.427577,-0.506131
4,600039015048,HC_train,1,11.0,600039015048,0.492985,0.456270,0.445612,0.430054,0.530365,...,-0.403509,-0.385902,-0.413289,-0.459313,-0.399568,-0.387467,-0.368935,-0.375250,-0.398979,-0.477383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,609910470154,smry_add,0,19.0,609910470154,0.470289,0.460937,0.413662,0.417567,0.478178,...,-0.403931,-0.405431,-0.416981,-0.408175,-0.386828,-0.403334,-0.374796,-0.380701,-0.407016,-0.501234
942,609919910881,smry_odd,1,15.0,609919910881,0.483536,0.431535,0.396026,0.376200,0.464298,...,-0.440654,-0.447909,-0.475961,-0.461891,-0.443016,-0.430285,-0.455808,-0.427278,-0.430173,-0.541160
943,609987705265,HC,1,15.0,609987705265,0.546239,0.524109,0.491601,0.512870,0.509529,...,-0.449078,-0.443418,-0.436006,-0.443410,-0.483405,-0.469088,-0.432302,-0.407773,-0.396675,-0.539863
944,609998081561,smry_add,1,12.0,609998081561,0.509430,0.453097,0.442876,0.445781,0.534367,...,-0.383580,-0.396591,-0.459420,-0.432853,-0.399698,-0.403501,-0.423135,-0.388684,-0.476045,-0.497686


In [11]:
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

def qudratic_r_squared_gender(chro_age, brain_age, gender):
    """
    compute the r squared of curve fit of chro_age vs. brain age.
    1. Use qudratic function to fit brain age with chronological age and gender.
    2. Then, compute r squared of brain age and predicted brain age with qudratic fit model.
    """
    
    def qudratic_fun(x, a, b, c, d, e, f):
        x1, x2 = x
        #return a + b*x1 + c*(x1**2) + d*x2 + e*x1*x2 f*(x1**2)*x2
        return a + b*x1 + c*(x1**2) + d*x2 + e*x1*x2 + f*(x1**2)*x2
    
    popt, pcov = curve_fit(qudratic_fun, (chro_age, gender), brain_age)
    brain_age_pred = qudratic_fun((chro_age, gender), popt[0], popt[1], popt[2], popt[3], popt[4], popt[5])

    r_square = r2_score(brain_age, brain_age_pred)
    return(r_square)
    

In [12]:
# train SVR with healthy control:
# from sklearn.model_selection import GridSearchCV
# from sklearn.svm import SVR
# import numpy as np

# C = np.power(10, np.linspace(-1,2, num = 20))
# epsilon=np.linspace(.001, 1, num = 20)

# param_grid_svr = dict(C = C, epsilon = epsilon)
# # linear model takes very long time to run.
# #grid_svr = GridSearchCV(estimator=SVR(kernel='linear'), param_grid=param_grid_svr, n_jobs=3, refit = True)
# # use default kernel rbf:
# grid_svr = GridSearchCV(estimator=SVR(), param_grid=param_grid_svr, n_jobs=3, refit = True)


###############################################ridge:#########################################
from sklearn.linear_model import RidgeCV
import numpy as np

# ridge regression:
n_alphas = 200
alphas = np.logspace(0, 3, n_alphas)
# print(alphas)
ridge_cv = RidgeCV(alphas=alphas, cv = 3)



In [13]:
from sklearn import preprocessing

test_list = disorders + ['HC']

plot_data_col = ['cluster', 'SUBJID', 'Sex', 'group', 'chronological age', 'brain age']

# create empty dataframe to save results:
column_index = pd.MultiIndex.from_product([['Pearson r', 'R square', 'MAE', 'rmse'],
                                           test_list])

row_index = np.sort(cluster_data_gmm[cluster_col_index].unique())
result_table = pd.DataFrame(index = row_index, columns = column_index)
result_table = pd.concat([result_table, pd.DataFrame(index = ['all'], columns = column_index)])

plot_data_list = []
# subjid = behav_feature['SUBJID'].values

# create feature list (features in each cluster:)
feature_list = []

hc_train_rows = behav_feature_group['group'] == 'HC_train'
y_train = behav_feature_group.loc[hc_train_rows, 'age_at_cnb'].values

for icluster in row_index:
    print(icluster)
    
    ifeature = cluster_data_gmm.loc[(cluster_data_gmm[cluster_col_index]==icluster) & 
                                    (cluster_data_gmm['modality']!='behav'), 'feature']    

    feature_list.append([ifeature, icluster])
    
## feature with all clusters:
# ifeature = cluster_data_gmm.loc[(cluster_data_gmm[cluster_col_index]<=10) & 
#                                     (cluster_data_gmm['modality']!='behav'), 'feature']

## feature with all featrues (include features removed in cluster analysis):
ifeature = cluster_data.loc[(cluster_data['modality']!='behav'), 'feature']

feature_list.append([ifeature, 'all'])



for i_feature, icluster in feature_list: 
    print('cluster:', icluster)
    
    X_train = pd.concat((behav_feature_group.loc[hc_train_rows, 'Sex'], 
                         behav_feature_group.loc[hc_train_rows, i_feature]),
                        axis = 1, sort = False).values
    
    print(X_train.shape)    
    # normalize X_train and X_test based on mean and sd of X_train. 
    # don't normalize 1st col which is sex, 
    scaler = preprocessing.StandardScaler().fit(X_train[:,1:])
    X_train[:,1:] = scaler.transform(X_train[:,1:])  
    
    seed = 7
    np.random.seed(seed)  
    
#     fit_result = grid_svr.fit(X_train, y_train)
#     print("Best: %f using %s" % (fit_result.best_score_, fit_result.best_params_))
    
    #fit_result = ridge_cv.fit(X_train, y_train)
    
    # train for male, female separately:
    X_train_male = X_train[X_train[:,0]==1, ]
    X_train_female = X_train[X_train[:,0]==0, ]
    
    fit_male = ridge_cv.fit(X_train_male[:,1:], y_train[X_train[:,0]==1])
    fit_female = ridge_cv.fit(X_train_female[:, 1:], y_train[X_train[:,0]==0])
    
    for i_dis in test_list:
        print('disorder:', i_dis)
        
        disorder_rows = behav_feature_group['group'] == i_dis
        y_test = behav_feature_group.loc[disorder_rows, 'age_at_cnb'].values
        subjid_test = behav_feature_group.loc[disorder_rows, 'SUBJID']

        X_test = pd.concat((behav_feature_group.loc[disorder_rows, 'Sex'], 
                            behav_feature_group.loc[disorder_rows, i_feature]),
                           axis = 1, sort = False).values

        X_test[:,1:] = scaler.transform(X_test[:,1:])
        
        #y_prediction = grid_svr.best_estimator_.predict(X_test)
        #y_prediction = ridge_cv.predict(X_test)
    
        # test for male and female separately:
        y_prediction = np.empty(X_test.shape[0])
        y_prediction[X_test[:,0]==1] = fit_male.predict(X_test[X_test[:,0]==1,1:])
        y_prediction[X_test[:,0]==0] = fit_female.predict(X_test[X_test[:,0]==0,1:])

        #result_table['R square', 'boot' + str(i)][icluster] = r2_score(y_test, y_prediction)
        #result_table['R square', 'boot' + str(i)][icluster] = qudratic_r_squared(y_test, y_prediction)
        result_table['R square', i_dis][icluster] = qudratic_r_squared_gender(y_test, y_prediction, X_test[:,1])
        result_table['Pearson r', i_dis][icluster] = np.corrcoef(y_test, y_prediction)[0,1]
        result_table['rmse', i_dis][icluster] = np.sqrt(np.mean(np.square(y_test - y_prediction)))
        result_table['MAE', i_dis][icluster] = np.mean(np.abs(y_test - y_prediction))

        plot_data_cv = pd.DataFrame(columns = plot_data_col)
        plot_data_cv['SUBJID'] = subjid_test
        plot_data_cv['chronological age'] = y_test
        plot_data_cv['brain age'] = y_prediction
        plot_data_cv.loc[:,'group'] = i_dis
        plot_data_cv.loc[:,'cluster'] = icluster
        plot_data_cv['Sex'] = X_test[:,0]

        plot_data_list.append(plot_data_cv)

plot_data = pd.concat(plot_data_list)
        

1.0
2.0
3.0
4.0
cluster: 1.0
(96, 80)
disorder: smry_ptd
disorder: smry_dep
disorder: smry_phb
disorder: smry_soc
disorder: smry_add
disorder: smry_odd
disorder: HC
cluster: 2.0
(96, 12)
disorder: smry_ptd
disorder: smry_dep
disorder: smry_phb
disorder: smry_soc
disorder: smry_add
disorder: smry_odd
disorder: HC
cluster: 3.0
(96, 56)
disorder: smry_ptd
disorder: smry_dep
disorder: smry_phb
disorder: smry_soc
disorder: smry_add
disorder: smry_odd
disorder: HC
cluster: 4.0
(96, 15)
disorder: smry_ptd
disorder: smry_dep
disorder: smry_phb
disorder: smry_soc
disorder: smry_add
disorder: smry_odd
disorder: HC
cluster: all
(96, 167)
disorder: smry_ptd
disorder: smry_dep
disorder: smry_phb
disorder: smry_soc
disorder: smry_add
disorder: smry_odd
disorder: HC


In [14]:
result_table.to_csv(output_dir+"/out03_result_table_ridge_disorder_" + cluster_col_index + ".csv")
result_table

Pearson r                                                              \
     smry_ptd  smry_dep  smry_phb  smry_soc  smry_add  smry_odd        HC   
1.0  0.688002  0.648755    0.7857  0.705944  0.719998  0.651907   0.79905   
2.0   0.40876  0.406445  0.618554  0.550602  0.573594  0.444068  0.643173   
3.0  0.580081  0.634647  0.687946  0.625654  0.723403  0.579297  0.808207   
4.0  0.279407  0.137664   0.24804  0.209643  0.327999   0.16842  0.423017   
all  0.697555  0.742904  0.801505  0.736895  0.803204  0.694907  0.812415   

     R square                       ...      MAE                       rmse  \
     smry_ptd  smry_dep   smry_phb  ... smry_add smry_odd       HC smry_ptd   
1.0  0.703949  0.642018   0.803056  ...  1.70014  1.63428  1.78788  1.99452   
2.0  0.481496  0.455758   0.597892  ...  2.53617  2.00393  2.75467  2.47846   
3.0  0.394507  0.622622   0.574963  ...  2.16344  2.29999  1.94989  2.63562   
4.0  0.167972  0.120122  0.0775829  ...  2.60118  2.19082   2.9247  2.62691   
all  0.570338  0.662208   0.737729  ...  1.82013  2.05198   1.9255  2.50954   

                                                           
    smry_dep smry_phb smry_soc smry_add smry_odd       HC  
1.0  1.90193  1.98803  2.07602   2.1337  2.02199  2.29863  
2.0  2.22932  3.04739  2.56779  3.07085  2.48081  3.22701  
3.0  2.18101  2.72137  2.61873  2.69457  2.77519    2.458  
4.0  2.57566  3.44482  2.88815  3.28343  2.70615  3.48636  
all  2.20541  2.38384  2.50906  2.28734  2.55098  2.55849  

[5 rows x 28 columns]

In [15]:
plot_data.to_csv(output_dir+"/out03_scatter_data_ridge_disorder_" + cluster_col_index + ".csv")
plot_data

,cluster,SUBJID,Sex,group,chronological age,brain age
26,1,600129552715,0.0,smry_ptd,11.0,14.409815
33,1,600210241146,0.0,smry_ptd,18.0,18.283338
70,1,600682103788,0.0,smry_ptd,18.0,16.869028
71,1,600689706588,0.0,smry_ptd,16.0,14.456132
80,1,600778703691,0.0,smry_ptd,13.0,12.954733
...,...,...,...,...,...,...
867,all,609173350200,1.0,HC,12.0,12.602509
920,all,609706993828,1.0,HC,10.0,10.463315
923,all,609714765360,1.0,HC,13.0,14.022208
931,all,609802779962,0.0,HC,16.0,18.360447
